# **Generating GVI Predictions from GSV Images: Comprehensive Workflow**

This colab illustrates the methodology to generate GSV images from a set of points and generate GVI predictions for them. We use the latitude and longitude of each point to query the Google Static API using custom parameters, and then feed the images into the Treepedia 2.0 ResNet model after appropriately resizing the images. The model outputs predictions that are saved to a CSV file by neighborhood for further analysis and visualization.

We encourage you to follow along to generate your own predictions!


# 1. Mount Your Drive and Create the Correct Folders

First, create a main folder in your google drive for your project and save the sampled points in a file titled "[Neighborhood Name]_Coords.csv" within this folder. Additionally, [download](https://drive.google.com/open?id=1A9IoXdKYolJ3G8TdTrYaqHlh3rKZ5wo8) and save the "weights_test.hdf5" file and save it in this folder. If the hyperlink does not work the link to download the weights file can also be found in the [Treepedia 2.0 repository README file](https://github.com/billcai/treepedia_dl_public?tab=readme-ov-file).

Now, navigate into the main folder you created.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
%cd [MAIN_FOLDER_PATH]

# 2. Install Necessary Packages

In [ ]:
import pandas as pd
from PIL import Image
import numpy as np
import model_lib as treepedia_dl
from os import readlink
import requests
from PIL import Image
from io import BytesIO
import pandas as pd
from tqdm import tqdm

# 3. Query the GSV API for Images

Use the function below to query the GSV API for images. You must provide your Google API Key. For the code to work as expected, use image size "400x400".

In our work, we choose to use the following values (for a more comprehensive list see the [Google Streeview Static API](https://developers.google.com/maps/documentation/streetview/request-streetview) page):

*   **Camera Heading:** Set to 0 degrees to standardize image
orientation and minimize directional bias
*   **Pitch:** Set to 0 degrees to ensure the camera
was directed straight ahead, parallel to the ground,
allowing us to capture the landscape’s greenery without introducing vertical distortions
* **Field of View:** Set to 90 degrees to provide a wide perspective of each
streetscape while maintaining proportions



In [ ]:
# Possible neighborhood names: Canton, Midtown, Roland_Park, Southwest_Baltimore
# pass in false for fetch_images first to make sure that all coordinates work
def readInNeighborhoodImages(neighborhood_name, api_key, size, heading, pitch, fov, fetch_images):
  coords_df = pd.read_csv(f'[MAIN_FOLDER_PATH]/{neighborhood_name} Coords.csv').round(6)
  img_paths = []
  num_not_available = 0
  for index, row in tqdm(coords_df.iterrows(), total=coords_df.shape[0]):
      latitude = row['y']  # Adjust column name as necessary
      longitude = row['x']  # Adjust column name as necessary

      # Metadata URL
      metadata_url = f"https://maps.googleapis.com/maps/api/streetview/metadata?size={size}&location={latitude},{longitude}&heading={heading}&pitch={pitch}&fov={fov}&key={api_key}"

      # Send request for metadata
      metadata_response = requests.get(metadata_url)
      metadata = metadata_response.json()

      if metadata['status'] == 'OK':
          # Construct URL to access the image
          img_paths.append(f'[MAIN_FOLDER_PATH]/{neighborhood_name}_{latitude}_{longitude}.jpg')
          image_url = f"https://maps.googleapis.com/maps/api/streetview?size={size}&location={latitude},{longitude}&heading={heading}&pitch={pitch}&fov={fov}&key={api_key}"

          # # Fetch and save the image
          if (fetch_images):
            image_response = requests.get(image_url)
            if image_response.status_code == 200:
                img = Image.open(BytesIO(image_response.content))
                img.save(f'[MAIN_FOLDER_PATH]/{neighborhood_name}_{latitude}_{longitude}.jpg')
      else:
          num_not_available += 1
          print(f"No imagery available for location: {latitude}, {longitude} - Status: {metadata['status']}")
  print('total not available', num_not_available)
  paths_df = pd.DataFrame(img_paths, columns=['ImagePath'])
  paths_df.to_csv(f'[MAIN_FOLDER_PATH]/{neighborhood_name}ImgPaths.csv',  index=False)

# 5. Generating Predictions for Neighborhood Images

Now that you have gathered your GSV images, we will will walk through how to use a pre-trained Treepedia model to generate predictions for images of a neighborhood. We will load the model, resize the images, and combine the predictions with their corresponding coordinates. The steps are described in detail below for your understanding. To execute them, just call the `generatePredictions` function defined below!

**Step 1:** Load the Model


First, we need to load the pre-trained Treepedia model from a weights file `weights_test.hdf5`.


```
print("Loading model from weights_test.hdf5")
model = treepedia_dl.load_keras_mod("weights_test.hdf5")
```


**Step 2:** Call Function from Above to Load and Resize Images

Next, we will load the image paths from a CSV file `({neighborhood_name}ImgPaths.csv)`, resize the images to match the model's input size, and extract their coordinates from the filenames.



```
current_model = (224, 224, 3)
imgdata_array, coordinates = load_and_resize_from_csv(f'[MAIN_FOLDER_PATH]/{neighborhood_name}ImgPaths.csv', current_model)
```

**Step 3:** Generate Predictions

Now, we will use the loaded model to generate predictions for the resized images.



```
predictions = model.predict(x=imgdata_array, batch_size=1, verbose=1)
```


**Step 4:** Combine Predictions with Coordinates

Finally, we will combine the predictions with their corresponding coordinates and save the results to a CSV file `({neighborhood_name}_Prediction.csv)`.


```
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
df = pd.DataFrame(predictions, columns=[f'{neighborhood_name} Predictions'])
combined_df = pd.concat([df_coordinates, df], axis=1)
combined_df.to_csv(f'[MAIN_FOLDER_PATH]/{neighborhood_name}_Prediction.csv', index=False)

```




In [ ]:
# Helper function to preprocess GSV Images; This function is called by generatePredictions
def load_and_resize_from_csv(csv_loc, current_model):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_loc)
    test_data = df.iloc[:, 0].tolist()  # Assuming image paths are in the first column

    imgdata = []
    coordinates = []

    for path in test_data:
        img = Image.open(path)
        img_resized = img.resize(current_model[:2])  # Resize according to the first two elements of current_model
        imgdata.append(img_resized)
        # Extract latitude and longitude from the filename
        filename = path.split('/')[-1]  # Adjust the split based on your path format
        parts = filename[:-4].split('_')  # Removes the file extension and splits at '_'
        latitude, longitude = float(parts[-2]), float(parts[-1])
        coordinates.append((latitude, longitude))

    # Convert the list of PIL images to a numpy array with the appropriate shape
    imgdata_array = np.zeros((len(imgdata), *current_model[:2], 3))  # Adjust the shape as necessary
    for i, img in enumerate(imgdata):
        imgdata_array[i] = np.array(img)

    return imgdata_array, coordinates

In [ ]:
def generatePredictions(neighborhood_name):
  print("Loading model from weights_test.hdf5")
  model = treepedia_dl.load_keras_mod("weights_test.hdf5")
  current_model = (224, 224, 3)
  imgdata_array, coordinates = load_and_resize_from_csv(f'[MAIN_FOLDER_PATH]/{neighborhood_name}ImgPaths.csv', current_model)
  df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
  print('Generating Predictions')
  predictions = model.predict(x=imgdata_array,batch_size=1,verbose=1)

  df = pd.DataFrame(predictions, columns=[f'{neighborhood_name} Predictions'])
  combined_df = pd.concat([df_coordinates, df], axis=1)

  combined_df.to_csv(f'[MAIN_FOLDER_PATH]/{neighborhood_name}_Prediction.csv', index=False)
  return combined_df

To generate predictions and save the predictions to a csv just run the next cell after replacing the relevant variables.

In [ ]:
generatePredictions('[Neighborhood_Name]').to_csv(f'[MAIN_FOLDER_PATH]/[Neighborhood_Name]_Prediction.csv', index=False)

# 6. You are Done!
You have now successfully generated predictions for images of a neighborhood using the Treepedia 2.0 model. Feel free to explore the predictions and use them for further analysis or visualization!